In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [13]:
data = pd.read_csv("../input/dl-course-data/ion.csv", index_col=0)
data.dropna(axis=0, subset=["Class"], inplace=True)
data["Class"] = data["Class"].map({"good": 1, "bad":0})

df_train = data.sample(frac=0.8, random_state=0)
df_valid = data.drop(df_train.index, axis=0)

max_ = df_train.max(axis=0)
min_ = df_train.min(axis=0)

df_train = (df_train - min_) / (max_ - min_)
df_valid = (df_valid - min_) / (max_ - min_)

x_train = df_train.drop(["Class"], axis=1)
x_valid = df_valid.drop(["Class"], axis=1)
y_train = df_train["Class"]
y_valid = df_valid["Class"]

In [17]:
from tensorflow import keras
from tensorflow.keras import layers, callbacks

earlt_stopping_callback = callbacks.EarlyStopping(
    min_delta=0.001, 
    patience=10,
    restore_best_weights=True)

model = keras.Sequential([
    layers.InputLayer(input_shape=[x_train.shape[1]]),
    layers.Dense(units=4, activation="relu"),
    layers.Dense(units=4, activation="relu"),
    layers.Dense(units=1, activation="sigmoid")
])

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["binary_accuracy"]
)

In [18]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_valid, y_valid),
    epochs=1000,
    batch_size=512,
    callbacks=[earlt_stopping_callback])

In [21]:
from IPython.display import display

history_df = pd.DataFrame(history.history)
print(history_df["val_loss"].min())
display(history_df.loc[:, ["loss", "val_loss"]].plot())
display(history_df.loc[:, ["binary_accuracy", "val_binary_accuracy"]].plot())